#Load files

#Libraries and packages


In [47]:
# Import pandas
import pandas as pd
import numpy as np

#-----------------------Libraries#------------------------  
# numpy
from numpy import mean
from numpy import std

from sklearn.model_selection import train_test_split

# matplotlib
import matplotlib.pyplot as plt

#------------------------ Models ------------------------ 
from sklearn import linear_model

from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
import xgboost

#Metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


#math
import math

In [48]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


Saving OSA_Regression.xlsx to OSA_Regression (1).xlsx
User uploaded file "OSA_Regression.xlsx" with length 28966 bytes


In [49]:

# Read OSA data into OSA_df
OSA_df = pd.read_excel("OSA_Regression.xlsx", sheet_name='Hoja1')


# 10 first instances of the data

In [50]:
OSA_df.head(10)

,IAH,Weight,Height,Cervical,Gender,Age,BMI
0,29.6,119,174,48.0,0,56,39.305060
1,19.7,78,168,42.0,0,39,27.636054
2,9.0,80,173,40.0,0,32,26.729927
3,2.0,109,190,42.0,0,32,30.193906
4,34.0,86,169,42.0,0,39,30.110991
5,60.0,145,172,44.0,0,47,49.012980
6,22.0,72,165,42.0,0,40,26.446281
7,11.0,80,180,38.0,0,28,24.691358
8,3.7,90,180,40.0,0,36,27.777778
9,7.0,50,158,35.0,1,50,20.028842


In [51]:
OSA_df.describe()

,IAH,Weight,Height,Cervical,Gender,Age,BMI
count,599.000000,599.000000,599.000000,599.000000,599.000000,599.000000,599.000000
mean,20.147346,87.704508,171.168614,40.631886,0.285476,49.502504,29.891201
std,18.144753,18.412257,9.502857,3.969171,0.452018,12.349546,5.630683
min,0.000000,45.000000,144.000000,30.000000,0.000000,20.000000,18.590125
25%,6.400000,75.000000,165.000000,38.000000,0.000000,40.000000,26.070325
50%,14.300000,86.000000,171.000000,41.000000,0.000000,49.000000,28.734672
75%,30.000000,98.000000,178.000000,43.000000,1.000000,59.000000,32.770513
max,108.400000,165.000000,197.000000,53.000000,1.000000,88.000000,63.654952


#Seperate X and y for train and test

In [52]:
features = ['Weight', 'Height', 'Cervical', 'BMI', 'Age']

X = np.array(OSA_df[features])

y = np.array(OSA_df['IAH'])

print('X shape:', X.shape, 'y shape: ', y.shape)

X shape: (599, 5) y shape:  (599,)


#Splitting test and training data 

In [53]:

## Example: Train 80% and Test 20%

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the shapes of both sets.
print(X_train.shape)
print(X_test.shape)

(479, 5)
(120, 5)


#Training (linear)

In [62]:
# Fitting a linear regression

# Import the linear models.


# Initialize the model class.

#model= linear_model.LinearRegression()

#model= linear_model.Ridge(alpha = 0.5)
model= linear_model.Lasso(alpha = 100)
#model = DecisionTreeRegressor(min_samples_split=4,max_depth=1)
#model = KNeighborsRegressor(n_neighbors=20)
#model = XGBRegressor(max_depth=1,min_child_weight=3)
# Fit the model to the training data.
Trained_model=model.fit(X_train, y_train)


params = {
    "n_estimators": 500,
    "max_depth": 4,
    "min_samples_split": 5,
    "learning_rate": 0.01,
    "loss": "squared_error",
}


#Testing

In [65]:
# Generate our predictions for the test set.
predictions = Trained_model.predict(X_test)
#predictions = grid_resultat.predict(X_test)


# Compute error between our test predictions and the actual values.
MSE=mean_squared_error(predictions, y_test)

RMSE=math.sqrt(mean_squared_error(predictions, y_test))

MAE=mean_absolute_error(predictions, y_test)

Rsquared = r2_score(y_test, predictions)

print('Mean squared Error: ', MSE,
      '\nMean Absolute Error: ', MAE,
      '\nR-squarred', Rsquared,
      '\nRMS', RMSE)


Mean squared Error:  326.1621103718769 
Mean Absolute Error:  14.280566630138638 
R-squarred 0.03352497386516162 
RMS 18.059958758864234


#KNN Nested

In [ ]:

#model = Ridge(alpha=1) , alpha = [0.001, 0.01, 0.1, 1, 10, 100, 1000] , param_grid = dict(alpha=alpha)


# configure the cross-validation procedure
cv_outer = KFold(n_splits=10, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
top_param =  list()
for train_ix, test_ix in cv_outer.split(X):
	# split data
  X_train, X_test = X[train_ix, :], X[test_ix, :]
  y_train, y_test = y[train_ix], y[test_ix]
	# configure the cross-validation procedure
  cv_inner = KFold(n_splits=3, shuffle=True, random_state=1)
	# define the model
  model = KNeighborsRegressor()
	# define search space
  #hyp_param = {"splitter":["best","random"],
                #"max_depth" : [1,3,5],
                #"min_samples_leaf":[1,2,3,4],}
  kn = {"n_neighbors":[20,50,100,200],}
	# define search
  search = GridSearchCV(model,param_grid=kn,scoring='neg_mean_squared_error',
cv=3,verbose=3)
	# execute search
  result = search.fit(X_train, y_train)
	# get the best performing model fit on the whole training set
  #print(result.best_estimator_)
  #print("best parameter ", result.best_params_)
  #print("best score ", result.best_score_)
	# store the result
  outer_results.append(result.best_score_)
  top_param.append(result.best_params_)
	# report progress
  
print(mean(outer_results))
print(top_param)
# summarize the estimated performance of the model
#print('Accuracy: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV 1/3] END .................n_neighbors=20;, score=-205.169 total time=   0.0s
[CV 2/3] END .................n_neighbors=20;, score=-367.415 total time=   0.0s
[CV 3/3] END .................n_neighbors=20;, score=-244.987 total time=   0.0s
[CV 1/3] END .................n_neighbors=50;, score=-195.589 total time=   0.0s
[CV 2/3] END .................n_neighbors=50;, score=-381.861 total time=   0.0s
[CV 3/3] END .................n_neighbors=50;, score=-244.302 total time=   0.0s
[CV 1/3] END ................n_neighbors=100;, score=-212.325 total time=   0.0s
[CV 2/3] END ................n_neighbors=100;, score=-391.651 total time=   0.0s
[CV 3/3] END ................n_neighbors=100;, score=-249.716 total time=   0.0s
[CV 1/3] END ................n_neighbors=200;, score=-225.309 total time=   0.0s
[CV 2/3] END ................n_neighbors=200;, score=-409.801 total time=   0.0s
[CV 3/3] END ................n_neighbors=200;, sc

#Ridge pick

In [56]:

#model = Ridge(alpha=1) , alpha = [0.001, 0.01, 0.1, 1, 10, 100, 1000] , param_grid = dict(alpha=alpha)


# configure the cross-validation procedure
cv_outer = KFold(n_splits=10, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
top_param =  list()
for train_ix, test_ix in cv_outer.split(X):
	# split data
  X_train, X_test = X[train_ix, :], X[test_ix, :]
  y_train, y_test = y[train_ix], y[test_ix]
	# configure the cross-validation procedure
  cv_inner = KFold(n_splits=3, shuffle=True, random_state=1)
	# define the model
  model = Ridge()
	# define search space
  alpha = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
  param_grid = dict(alpha=alpha)

	# define search
  search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='r2', verbose=1,
n_jobs=-1)
	# execute search
  result = search.fit(X_train, y_train)
	# get the best performing model fit on the whole training set
  #print(result.best_estimator_)
  #print("best parameter ", result.best_params_)
  #print("best score ", result.best_score_)
	# store the result
  outer_results.append(result.best_score_)
  top_param.append(result.best_params_)
	# report progress
  
print(mean(outer_results))
print(top_param)
# summarize the estimated performance of the model
#print('Accuracy: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))


Fitting 5 folds for each of 7 candidates, totalling 35 fits
Fitting 5 folds for each of 7 candidates, totalling 35 fits
Fitting 5 folds for each of 7 candidates, totalling 35 fits
Fitting 5 folds for each of 7 candidates, totalling 35 fits
Fitting 5 folds for each of 7 candidates, totalling 35 fits
Fitting 5 folds for each of 7 candidates, totalling 35 fits
Fitting 5 folds for each of 7 candidates, totalling 35 fits
Fitting 5 folds for each of 7 candidates, totalling 35 fits
Fitting 5 folds for each of 7 candidates, totalling 35 fits
Fitting 5 folds for each of 7 candidates, totalling 35 fits
0.22022604344812735
[{'alpha': 100}, {'alpha': 10}, {'alpha': 10}, {'alpha': 10}, {'alpha': 100}, {'alpha': 10}, {'alpha': 100}, {'alpha': 10}, {'alpha': 100}, {'alpha': 100}]


#Lasso 

In [1]:

#model = Ridge(alpha=1) , alpha = [0.001, 0.01, 0.1, 1, 10, 100, 1000] , param_grid = dict(alpha=alpha)


# configure the cross-validation procedure
cv_outer = KFold(n_splits=10, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
top_param =  list()
for train_ix, test_ix in cv_outer.split(X):
	# split data
  X_train, X_test = X[train_ix, :], X[test_ix, :]
  y_train, y_test = y[train_ix], y[test_ix]
	# configure the cross-validation procedure
  cv_inner = KFold(n_splits=3, shuffle=True, random_state=1)
	# define the model
  model = Lasso()
	# define search space
  alpha = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
  param_grid = dict(alpha=alpha)

	# define search
  search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='r2', verbose=1,
n_jobs=-1)
	# execute search
  result = search.fit(X_train, y_train)
	# get the best performing model fit on the whole training set
  #print(result.best_estimator_)
  #print("best parameter ", result.best_params_)
  #print("best score ", result.best_score_)
	# store the result
  outer_results.append(result.best_score_)
  top_param.append(result.best_params_)
	# report progress
  
print(mean(outer_results))
print(top_param)
# summarize the estimated performance of the model
#print('Accuracy: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))


NameError: ignored

#Decision TREE

In [ ]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# configure the cross-validation procedure
cv_outer = KFold(n_splits=10, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in cv_outer.split(X):
	# split data
	X_train, X_test = X[train_ix, :], X[test_ix, :]
	y_train, y_test = y[train_ix], y[test_ix]
	# configure the cross-validation procedure
	cv_inner = KFold(n_splits=3, shuffle=True, random_state=1)
	# define the model
	model = DecisionTreeRegressor(random_state=1)
	# define search space

	tree_para = {'criterion':['gini','entropy'],'max_depth':[4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150]} # decision tree para

	# define search
	search = GridSearchCV(model, tree_para, scoring='recall', cv=cv_inner,refit = True, verbose = 3)
	# execute search
	result = search.fit(X_train, y_train)
	# get the best performing model fit on the whole training set
	best_model = result.best_estimator_
	# evaluate model on the hold out dataset
	yhat = best_model.predict(X_test)
	# evaluate the model
	acc = accuracy_score(y_test, yhat)
	# store the result
	outer_results.append(acc)
	# report progress
	print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))

Fitting 3 folds for each of 36 candidates, totalling 108 fits
[CV 1/3] END .........criterion=gini, max_depth=4;, score=nan total time=   0.0s
[CV 2/3] END .........criterion=gini, max_depth=4;, score=nan total time=   0.0s
[CV 3/3] END .........criterion=gini, max_depth=4;, score=nan total time=   0.0s
[CV 1/3] END .........criterion=gini, max_depth=5;, score=nan total time=   0.0s
[CV 2/3] END .........criterion=gini, max_depth=5;, score=nan total time=   0.0s
[CV 3/3] END .........criterion=gini, max_depth=5;, score=nan total time=   0.0s
[CV 1/3] END .........criterion=gini, max_depth=6;, score=nan total time=   0.0s
[CV 2/3] END .........criterion=gini, max_depth=6;, score=nan total time=   0.0s
[CV 3/3] END .........criterion=gini, max_depth=6;, score=nan total time=   0.0s
[CV 1/3] END .........criterion=gini, max_depth=7;, score=nan total time=   0.0s
[CV 2/3] END .........criterion=gini, max_depth=7;, score=nan total time=   0.0s
[CV 3/3] END .........criterion=gini, max_depth

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
108 fits failed out of a total of 108.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
54 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 1320, in fit
    X_idx_sorted=X_idx_sorted,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 356, in fit
    criterion = CRITERIA_REG[self.criterion](self.n_outputs_, n_samples)
KeyError: 'gini'

-----------

KeyError: ignored

#SVM

In [ ]:



#model = Ridge(alpha=1) , alpha = [0.001, 0.01, 0.1, 1, 10, 100, 1000] , param_grid = dict(alpha=alpha)


# configure the cross-validation procedure
cv_outer = KFold(n_splits=3, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
top_param =  list()
for train_ix, test_ix in cv_outer.split(X):
	# split data
  X_train, X_test = X[train_ix, :], X[test_ix, :]
  y_train, y_test = y[train_ix], y[test_ix]
	# configure the cross-validation procedure
  cv_inner = KFold(n_splits=3, shuffle=True, random_state=1)
	# define the model
  model = XGBRegressor()
	# define search space
  params = {'max_depth':[1,2,3,4,5,6,7,8,9,10],'min_child_weight':[1,2,3,4,5]}

	# define search
  search = GridSearchCV(estimator=model, param_grid=params, scoring='neg_mean_absolute_error', verbose=1,
n_jobs=-1)
	# execute search
  result = search.fit(X_train, y_train)
	# get the best performing model fit on the whole training set
  #print(result.best_estimator_)
  #print("best parameter ", result.best_params_)
  #print("best score ", result.best_score_)
	# store the result
  outer_results.append(result.best_score_)
  top_param.append(result.best_params_)
	# report progress
  
print(mean(outer_results))
print(top_param)
# summarize the estimated performance of the model
#print('Accuracy: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))


Fitting 5 folds for each of 50 candidates, totalling 250 fits
[04:13:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 50 candidates, totalling 250 fits
[04:14:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 50 candidates, totalling 250 fits
[04:14:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
-12.170021749089841
[{'max_depth': 1, 'min_child_weight': 5}, {'max_depth': 1, 'min_child_weight': 4}, {'max_depth': 1, 'min_child_weight': 3}]
